In [1]:
import pandas as pd
df = pd.read_csv('data/spotify_2023_clean.csv', encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   track_name         953 non-null    object
 1   artist             953 non-null    object
 2   featuring_artists  366 non-null    object
 3   artist_count       953 non-null    int64 
 4   streams            953 non-null    int64 
 5   bpm                953 non-null    int64 
 6   key                858 non-null    object
 7   mode               953 non-null    object
 8   danceability       953 non-null    int64 
 9   valence            953 non-null    int64 
 10  energy             953 non-null    int64 
 11  acousticness       953 non-null    int64 
 12  instrumentalness   953 non-null    int64 
 13  liveness           953 non-null    int64 
 14  speechiness        953 non-null    int64 
 15  released_date      953 non-null    object
dtypes: int64(10), object(6)
memory usage: 119.3+

In [2]:
# Stream values hard to read
df['streams'] = df['streams'].astype(float)
pd.options.display.float_format = '{:,.0f}'.format

In [3]:
# GROUP BY ARTISTS
groups = df.groupby('artist')
# for name, group in groups:
#     print('Artist:', name)
#     print(group.loc[:,['track_name', 'artist', 'streams']])
#     print('\n')

In [4]:
# 1. Look at the Taylor Swift section
groups.get_group('Taylor Swift').head()

,track_name,artist,featuring_artists,artist_count,streams,bpm,key,mode,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness,released_date
3,Cruel Summer,Taylor Swift,NaN,1,"800,840,817",170,A,Major,55,58,72,11,0,11,15,2019-08-23
21,I Can See You (TaylorÃ¯Â¿Â½Ã¯Â¿Â½Ã¯Â¿Â½s Versi...,Taylor Swift,NaN,1,"52,135,248",123,F#,Major,69,82,76,6,0,6,3,2023-07-07
33,Anti-Hero,Taylor Swift,NaN,1,"999,748,277",97,E,Major,64,51,63,12,0,19,5,2022-10-21
37,Blank Space,Taylor Swift,NaN,1,"1,355,959,075",96,F,Major,75,57,68,9,0,13,6,2014-01-01
38,Style,Taylor Swift,NaN,1,"786,181,836",95,D,Major,60,48,79,0,0,12,4,2014-01-01


In [5]:
# 2. The number of songs of each artist
# groups.size().sort_values(ascending=False).head()
pd.DataFrame(groups.size()).sort_values(by=0, ascending=False).head()

,0
artist,
Taylor Swift,36
The Weeknd,34
Bad Bunny,26
Kendrick Lamar,23
SZA,23


In [6]:
# 3. The number artists who has more than 10 songs
temp_df = pd.DataFrame(groups.size())
temp_df.rename(columns={0:'count'}, inplace=True)
# temp_df[temp_df['count'] > 10].sort_values(by='count', ascending=False)    # to see whole list
temp_df[temp_df['count'] > 10].shape[0]

10

In [7]:
# 4. Top 10 artist by average streams - This is not meaningful expression because of the calculation of mean
groups.mean('streams').nlargest(10, 'streams')[['streams']]

,streams
artist,
Tones and I,"2,864,791,672"
Glass Animals,"2,557,975,762"
The Chainsmokers,"2,397,652,496"
Queen,"2,197,010,679"
Hozier,"2,135,158,446"
Vance Joy,"2,009,094,673"
Ray Dalton,"1,953,533,826"
Tom Odell,"1,813,673,666"
The Killers,"1,806,617,704"


In [8]:
# 5. Top 10 artist by total streams
# groups.sum('streams').nlargest(10, 'streams').loc[:,'streams']
groups.sum('streams').nlargest(10, 'streams')[['streams']]

,streams
artist,
The Weeknd,"21,516,545,916"
Bad Bunny,"15,363,784,442"
Ed Sheeran,"14,559,679,731"
Taylor Swift,"14,423,238,266"
Harry Styles,"11,608,645,649"
Eminem,"10,193,727,260"
Justin Bieber,"8,138,018,785"
Drake,"8,043,031,261"
Imagine Dragons,"7,719,446,948"


In [9]:
# 6. Top 10 artist by total streams see their average characteristics
wanted = {'streams':'sum',
          'danceability': 'mean',
          'valence': 'mean',
          'energy': 'mean',
          'acousticness': 'mean',
          'instrumentalness': 'mean',
          'liveness': 'mean',
          'speechiness': 'mean'
         }
groups.agg(wanted).nlargest(10, 'streams').loc[:,'streams':'speechiness']

,streams,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
artist,,,,,,,,
The Weeknd,"21,516,545,916",60,43,64,21,1,21,8
Bad Bunny,"15,363,784,442",74,49,67,24,2,20,10
Ed Sheeran,"14,559,679,731",71,53,63,34,0,16,5
Taylor Swift,"14,423,238,266",60,34,56,28,1,16,7
Harry Styles,"11,608,645,649",61,54,59,43,2,14,5
Eminem,"10,193,727,260",80,47,74,6,0,23,16
Justin Bieber,"8,138,018,785",66,60,63,35,0,23,14
Drake,"8,043,031,261",74,31,55,6,0,23,20
Imagine Dragons,"7,719,446,948",66,58,74,15,0,28,14


In [10]:
# GROUP BY RELEASE YEAR
df['released_date'] = pd.to_datetime(df['released_date'])
df['year'] = df['released_date'].dt.year
yearly_groups = df.groupby('year')
# for year, group in yearly_groups:
#     print('Year:', year)
#     print(group.loc[:,['track_name', 'artist', 'streams']])
#     print('\n')

In [11]:
# 7. Number of songs by release year
# groups.size().sort_values(ascending=False).head()
pd.DataFrame(yearly_groups.size()).sort_values(by=0, ascending=False).head()

,0
year,
2022,402
2023,175
2021,119
2020,37
2019,36


In [12]:
# 8. Number of total listens by release year
yearly_groups.sum('streams').nlargest(5, 'streams')[['streams']]

,streams
year,
2022,"117,138,011,949"
2021,"73,808,103,039"
2019,"37,870,932,581"
2020,"34,703,731,857"
2017,"31,457,639,060"


In [13]:
# 9. Top 3 most listened artists for songs released in 2023
df_2023 = yearly_groups.get_group(2023)
artist_groups = df_2023.groupby('artist')
artist_groups.sum('streams').nlargest(3, 'streams')[['streams']]

,streams
artist,
Karol G,"1,354,836,810"
Miley Cyrus,"1,316,855,716"
Bad Bunny,"1,059,213,008"


In [14]:
# GROUP BY RELEASE MONTH
df['released_date'] = pd.to_datetime(df['released_date'])
df['month'] = df['released_date'].dt.month
monthly_groups = df.groupby('month')
# for month, group in monthly_groups:
#     print('Month:', month)
#     print(group.loc[:,['track_name', 'artist', 'streams']])
#     print('\n')

In [15]:
# 10. Number of songs which released in same month
pd.DataFrame(monthly_groups.size()).sort_values(by=0, ascending=False).head()

,0
month,
1,134
5,128
3,86
6,86
11,80


In [16]:
# 11. Number of total listens of songs released in same month
monthly_groups.sum('streams').nlargest(5, 'streams')[['streams']]

,streams
month,
1,"96,983,116,526"
5,"53,205,638,976"
11,"44,207,381,649"
10,"42,989,875,350"
9,"41,140,083,774"


In [17]:
# 12. Total streams and average characteristics by release month
wanted = {'streams':'sum',
          'danceability': 'mean',
          'valence': 'mean',
          'energy': 'mean',
          'acousticness': 'mean',
          'instrumentalness': 'mean',
          'liveness': 'mean',
          'speechiness': 'mean'
         }
monthly_groups.agg(wanted).loc[:,'streams':'speechiness']

,streams,danceability,valence,energy,acousticness,instrumentalness,liveness,speechiness
month,,,,,,,,
1,"96,983,116,526",66,57,66,24,1,18,8
2,"21,542,363,634",67,58,66,29,1,19,11
3,"41,026,524,341",67,54,68,22,2,20,10
4,"26,673,634,664",68,47,62,27,0,17,10
5,"53,205,638,976",69,51,63,33,2,18,12
6,"35,263,841,711",72,51,65,23,3,19,10
7,"29,894,941,615",67,53,65,24,3,16,8
8,"29,038,222,245",67,49,67,24,1,17,12
9,"41,140,083,774",68,49,65,25,1,19,11
